# Building an AI Research Agent for Image Analysis

## Overview

This comprehensive guide will teach you how to build an AI research agent capable of conducting in-depth research based on image analysis. Using the **[Granite 3.2 Vision Model](https://github.com/ibm-granite-community)** alongside the **[Granite 3.2 8B Language Model](https://github.com/ibm-granite-community)** with enhanced reasoning capabilities, you'll learn to create an advanced image researcher that runs entirely locally using [Ollama](https://ollama.ai), [Open WebUI](https://github.com/open-webui/open-webui), and [Granite](https://github.com/ibm-granite-community).

### What You'll Learn

- Understanding Vision Models and their capabilities
- Building multi-agent systems with [CrewAI](https://docs.crewai.com)
- Implementing RAG (Retrieval-Augmented Generation) for research
- Orchestrating parallel, asynchronous research tasks
- Processing images and extracting researchable concepts
- Synthesizing findings into comprehensive reports
- Handling edge cases and optimizing agent performance
- Troubleshooting common issues in agentic AI systems

### What is Agentic AI for Image Analysis?

**Agentic AI for Image Analysis** refers to AI systems that can:
- **Perceive**: Analyze images using vision models to extract visual information
- **Plan**: Identify researchable concepts and create structured research plans
- **Research**: Conduct parallel research using RAG to gather information from web and documents
- **Synthesize**: Combine findings into comprehensive, well-cited reports
- **Learn**: Improve research quality through better prompt engineering and tool usage

### Use Cases

- **Architecture Diagrams**: Understand components, protocols, and system relationships
- **Business Dashboards**: Explain KPIs, metrics, and trends in BI tools
- **Artwork and Historical Photos**: Analyze artistic styles, historical context, and related works
- **Scientific Visualizations**: Interpret complex charts, lab results, or datasets
- **Medical Imaging**: Research conditions, treatments, and related medical information
- **Educational Content**: Extract and research concepts from educational diagrams

---

## Table of Contents

1. [Prerequisites](#prerequisites)
2. [Understanding Key Concepts](#understanding-key-concepts)
3. [Environment Setup](#environment-setup)
4. [Project Setup](#project-setup)
5. [Building the Vision Model Integration](#building-the-vision-model-integration)
6. [Creating the Research Item Identifier Agent](#creating-the-research-item-identifier-agent)
7. [Building the Research Crew](#building-the-research-crew)
8. [Integrating RAG Tools](#integrating-rag-tools)
9. [Putting It All Together](#putting-it-all-together)
10. [Advanced Techniques](#advanced-techniques)
11. [Edge Cases and Error Handling](#edge-cases-and-error-handling)
12. [Troubleshooting Guide](#troubleshooting-guide)
13. [Exercises](#exercises)
14. [Summary](#summary)


## Prerequisites

Before we begin, make sure you have:

1. **[Python 3.8+](https://www.python.org/downloads/)** installed on your system
2. **Basic knowledge** of [Python](https://www.python.org), object-oriented programming, and APIs
3. **Understanding** of LLMs and vision models (helpful but not required)
4. **[Ollama](https://ollama.ai)** installed and running locally
5. **Required packages** (we'll install these in the next section)

### Required Python Packages

- [`crewai`](https://github.com/joaomdmoura/crewAI) - Multi-agent orchestration framework
- [`crewai-tools`](https://github.com/joaomdmoura/crewai-tools) - Tools for CrewAI agents (web search, file operations, etc.)
- [`litellm`](https://docs.litellm.ai) - LLM provider abstraction (supports Ollama, OpenAI, etc.)
- [`pydantic`](https://docs.pydantic.dev) - Data validation using Python type annotations
- `base64` - Image encoding/decoding (built-in Python module)
- [`PIL` (Pillow)](https://pillow.readthedocs.io) - Image processing
- [`requests`](https://requests.readthedocs.io) - HTTP requests for API calls

### System Requirements

- **CPU**: Any modern CPU will work
- **GPU**: Optional but recommended for faster inference (CUDA-compatible GPU or Apple Silicon)
- **RAM**: At least 8GB recommended (16GB+ for better performance)
- **Storage**: ~10GB for models and dependencies
- **[Ollama](https://ollama.ai)**: Must be installed and running on localhost:11434

### Installing Ollama

If you haven't installed Ollama yet:

1. **macOS/Linux**: Visit [ollama.ai](https://ollama.ai) and download the installer
2. **Windows**: Download from [ollama.ai](https://ollama.ai)
3. **After installation**, pull the required models:
   ```bash
   ollama pull granite3.2:8b-instruct-q8_0
   ollama pull granite3.2-vision:2b
   ```

### Verifying Ollama Installation

Run this command to verify Ollama is working:
```bash
ollama list
```

You should see your models listed. If not, pull them using the commands above.


In [ ]:
# Install required packages (uncomment if needed)
# %pip install crewai crewai-tools litellm pydantic pillow requests

# Verify installation
try:
    import crewai
    import crewai_tools
    import litellm
    import pydantic
    from PIL import Image
    import base64
    import requests
    print("✅ All required packages are installed!")
    print(f"CrewAI version: {crewai.__version__ if hasattr(crewai, '__version__') else 'installed'}")
except ImportError as e:
    print(f"❌ Missing package: {e}")
    print("Please install missing packages using: pip install crewai crewai-tools litellm pydantic pillow requests")


## Understanding Key Concepts

Before diving into the implementation, let's understand the fundamental concepts:

### 1. Vision Models

**Vision Models** are AI models that can understand and describe images. They combine:
- **Image Encoders**: Convert images into numerical representations
- **Language Models**: Generate text descriptions from those representations

**Key Capabilities:**
- Object detection and recognition
- Scene understanding
- Text extraction from images (OCR)
- Relationship understanding between objects

**[Granite 3.2 Vision Model](https://github.com/ibm-granite-community):**
- 2B parameter model optimized for vision tasks
- Runs efficiently on local hardware
- Supports various image formats (PNG, JPEG, etc.)
- Can describe images in detail for educational/research purposes

### 2. Agentic AI with CrewAI

**[CrewAI](https://docs.crewai.com)** is a framework for orchestrating multiple AI agents to work together:

**Key Components:**
- **Agents**: Individual AI workers with specific roles and goals
- **Tasks**: Specific jobs assigned to agents
- **Crews**: Teams of agents working together
- **Tools**: External capabilities agents can use (web search, file operations, etc.)

**Process Types:**
- **Sequential**: Tasks run one after another
- **Hierarchical**: Tasks organized in a hierarchy
- **Parallel**: Multiple tasks run simultaneously (what we'll use for research)

### 3. Retrieval-Augmented Generation (RAG)

**RAG** combines:
- **Retrieval**: Finding relevant information from sources (web, documents, databases)
- **Augmentation**: Adding retrieved context to prompts
- **Generation**: Using LLMs to generate responses based on augmented context

**Why RAG?**
- ✅ Provides up-to-date information (not limited by training cutoff)
- ✅ Grounds responses in verifiable sources
- ✅ Reduces hallucinations
- ✅ Enables domain-specific knowledge

### 4. Parallel Research Architecture

Our system uses a **two-stage parallel architecture**:

1. **Stage 1 - Identification**: Single agent analyzes image and identifies research topics
2. **Stage 2 - Research**: Multiple agents research topics in parallel

**Benefits:**
- ⚡ Faster research (parallel execution)
- 🎯 Focused research (each agent handles one topic)
- 📊 Comprehensive coverage (all topics researched simultaneously)

### 5. Pydantic Models for Structured Output

**[Pydantic](https://docs.pydantic.dev)** provides data validation and structured output:

- Ensures LLM outputs match expected format
- Enables type-safe data handling
- Simplifies integration between agents

**In our system:**
- `ResearchItem`: Represents a single research topic
- `ResearchItems`: Collection of research items
- Used to pass structured data between agents


## Environment Setup

### Step 1: Verify Ollama is Running

First, let's verify that Ollama is running and accessible:


In [ ]:
import requests
import json

# Check if Ollama is running
OLLAMA_BASE_URL = "http://localhost:11434"

def check_ollama_connection():
    """Verify Ollama is running and accessible."""
    try:
        response = requests.get(f"{OLLAMA_BASE_URL}/api/tags", timeout=5)
        if response.status_code == 200:
            models = response.json().get("models", [])
            print("✅ Ollama is running!")
            print(f"\nAvailable models ({len(models)}):")
            for model in models[:5]:  # Show first 5 models
                print(f"  - {model.get('name', 'Unknown')}")
            if len(models) > 5:
                print(f"  ... and {len(models) - 5} more")
            
            # Check for required models
            required_models = ["granite3.2:8b-instruct-q8_0", "granite3.2-vision:2b"]
            available_model_names = [m.get('name', '') for m in models]
            
            print("\n📋 Required models:")
            for model_name in required_models:
                if any(model_name in name for name in available_model_names):
                    print(f"  ✅ {model_name} - Available")
                else:
                    print(f"  ❌ {model_name} - Not found (run: ollama pull {model_name})")
            
            return True
        else:
            print(f"❌ Ollama returned status code: {response.status_code}")
            return False
    except requests.exceptions.ConnectionError:
        print("❌ Cannot connect to Ollama. Please ensure:")
        print("  1. Ollama is installed")
        print("  2. Ollama is running (run 'ollama serve' in terminal)")
        print("  3. Ollama is accessible at http://localhost:11434")
        return False
    except Exception as e:
        print(f"❌ Error checking Ollama: {e}")
        return False

# Check connection
ollama_ready = check_ollama_connection()


## Project Setup

Now let's set up our project by importing all necessary libraries and configuring our LLM connections.


In [ ]:
# Import required libraries
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import WebsiteSearchTool, FileReadTool, DirectoryReadTool
from pydantic import BaseModel
from typing import List
import base64
from PIL import Image
import io
import os
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

# Display versions
try:
    import crewai
    print(f"CrewAI: {crewai.__version__ if hasattr(crewai, '__version__') else 'installed'}")
except:
    pass


### Step 1: Set Up Connections to the Two LLMs

We need two different LLMs for our system:

1. **[Granite Vision LLM](https://github.com/ibm-granite-community)** (`granite3.2-vision:2b`): Transforms images into descriptive text
2. **[Granite 3.2 Instruct](https://github.com/ibm-granite-community)** (`granite3.2:8b-instruct-q8_0`): Handles all language tasks including planning, reasoning, and synthesis

**Why q8_0 quantization?**
- Better tool calling reliability than q4_K_M
- Still lightweight enough to run locally
- Good balance between quality and performance

**Temperature = 0:**
- Ensures deterministic, consistent outputs
- Important for structured data extraction
- Reduces variability in research quality


In [ ]:
# Configure LLM connections
OLLAMA_BASE_URL = "http://localhost:11434"
OLLAMA_API_KEY = "ollama"  # Ollama doesn't require a real API key, but CrewAI expects one

# Main LLM for language tasks (planning, reasoning, synthesis)
llm = LLM(
    model="granite3.2:8b-instruct-q8_0",
    base_url=OLLAMA_BASE_URL,
    api_key=OLLAMA_API_KEY,
    temperature=0  # Deterministic outputs for structured data
)

# Vision LLM for image analysis
vision_llm = LLM(
    model="granite3.2-vision:2b",
    base_url=OLLAMA_BASE_URL,
    api_key=OLLAMA_API_KEY,
    temperature=0  # Consistent image descriptions
)

print("✅ LLM connections configured!")
print(f"Main LLM: granite3.2:8b-instruct-q8_0")
print(f"Vision LLM: granite3.2-vision:2b")
print(f"Base URL: {OLLAMA_BASE_URL}")


## Building the Vision Model Integration

### Image Processing Functions

Before we can analyze images, we need functions to:
1. Load images from files
2. Convert images to base64 format (required for vision models)
3. Call the vision model to get descriptions

Let's build these utilities:


In [ ]:
def image_to_base64(image_path: str) -> str:
    """
    Convert an image file to base64 encoded string.
    
    Args:
        image_path: Path to the image file
        
    Returns:
        Base64 encoded string with data URI prefix
    """
    try:
        with open(image_path, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
            # Determine file extension
            ext = os.path.splitext(image_path)[1].lower()
            mime_type = {
                '.png': 'image/png',
                '.jpg': 'image/jpeg',
                '.jpeg': 'image/jpeg',
                '.gif': 'image/gif',
                '.webp': 'image/webp'
            }.get(ext, 'image/png')
            
            return f"data:{mime_type};base64,{encoded_string}"
    except FileNotFoundError:
        raise FileNotFoundError(f"Image file not found: {image_path}")
    except Exception as e:
        raise Exception(f"Error encoding image: {e}")


def describe_image(image_path: str, query: str = "Describe the image in detail for educational purposes.") -> str:
    """
    Use the vision model to describe an image.
    
    Args:
        image_path: Path to the image file
        query: The prompt/question for the vision model
        
    Returns:
        Detailed description of the image
    """
    try:
        # Convert image to base64
        image_data = image_to_base64(image_path)
        
        # Prepare the message for the vision model
        # Note: The exact format depends on how CrewAI's LLM handles vision inputs
        # This is a simplified version - you may need to adjust based on your CrewAI version
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": query},
                    {"type": "image_url", "image_url": {"url": image_data}}
                ]
            }
        ]
        
        # Call the vision model
        # Note: This is a simplified call - actual implementation may vary
        # You might need to use litellm directly or adjust based on CrewAI's API
        response = vision_llm.call(messages)
        
        return response
    except Exception as e:
        raise Exception(f"Error describing image: {e}")


# Test the functions (uncomment when you have an image)
# test_image_path = "path/to/your/test/image.png"
# description = describe_image(test_image_path)
# print(description)

print("✅ Image processing functions defined!")
print("\nFunctions available:")
print("  - image_to_base64(image_path): Convert image to base64")
print("  - describe_image(image_path, query): Get image description from vision model")


In [ ]:
def describe_image_ollama_direct(image_path: str, query: str = "Describe the image in detail for educational purposes.") -> str:
    """
    Alternative: Use Ollama API directly for vision model calls.
    This is more reliable if CrewAI's LLM wrapper doesn't support vision well.
    
    Args:
        image_path: Path to the image file
        query: The prompt/question for the vision model
        
    Returns:
        Detailed description of the image
    """
    import requests
    
    try:
        # Convert image to base64
        image_data = image_to_base64(image_path)
        
        # Prepare the request payload for Ollama
        payload = {
            "model": "granite3.2-vision:2b",
            "prompt": query,
            "images": [image_data.split(',')[1]],  # Remove data URI prefix
            "stream": False
        }
        
        # Make the API call
        response = requests.post(
            f"{OLLAMA_BASE_URL}/api/generate",
            json=payload,
            timeout=120  # Vision models can take longer
        )
        
        if response.status_code == 200:
            result = response.json()
            return result.get("response", "No description generated.")
        else:
            raise Exception(f"Ollama API returned status {response.status_code}: {response.text}")
            
    except Exception as e:
        raise Exception(f"Error describing image: {e}")


print("✅ Alternative vision model function (Ollama direct) defined!")
print("Use describe_image_ollama_direct() if the CrewAI method doesn't work.")


## Creating the Research Item Identifier Agent

### Step 2: Define Pydantic Models for Structured Output

First, we need to define the data structures that will hold our research items. This ensures the LLM outputs structured, validated data.


In [ ]:
# Define Pydantic models for structured output
class ResearchItem(BaseModel):
    """
    Represents a single research item identified in an image.
    
    Attributes:
        item_name: The name/title of the item to research
        research_instructions: Specific instructions on what to research about this item
    """
    item_name: str
    research_instructions: str


class ResearchItems(BaseModel):
    """
    Container for multiple research items.
    
    Attributes:
        items: List of ResearchItem objects
    """
    items: List[ResearchItem]

print("✅ Pydantic models defined!")
print("\nModels:")
print("  - ResearchItem: Single research topic with name and instructions")
print("  - ResearchItems: Collection of research items")


### Step 3: Create the Item Identifier Agent

This agent analyzes the image description and user query to identify key features and concepts requiring research.

**Agent Role:**
- **Role**: Item Identifier
- **Goal**: Identify concepts in the image requiring research
- **Responsibilities**: 
  - Analyze image descriptions
  - Extract researchable concepts
  - Create structured research plans

**Inputs:**
- Image description generated by the Vision Model
- User's research goal or context (optional)

**Outputs:**
- Structured list of items for further research (as ResearchItems Pydantic model)


In [ ]:
# Create the Item Identifier Agent
item_identifier = Agent(
    role='Item Identifier',
    goal='Identify concepts in the image requiring research.',
    backstory="""You are an expert at analyzing images and identifying 
    key concepts, objects, and topics that would benefit from further research. 
    You break down complex images into researchable components.""",
    llm=llm,
    verbose=True
)

# Create the identification task
identification_task = Task(
    description="""Analyze the following image description and identify all 
    researchable items, concepts, or topics. For each item, provide:
    1. A clear name/title
    2. Specific research instructions on what to find out about it.
    
    Image Description: {image_description}
    User Query: {user_query}
    
    Focus on identifying distinct, researchable topics that would provide 
    educational value or answer the user's question.""",
    agent=item_identifier,
    expected_output='A structured list of items and concepts for further research, with clear research instructions for each.',
    output_pydantic=ResearchItems
)

# Create the identifier crew
identifier_crew = Crew(
    agents=[item_identifier],
    tasks=[identification_task],
    process=Process.sequential,
    verbose=True
)

print("✅ Research Item Identifier Agent created!")
print("\nComponents:")
print("  - Agent: Item Identifier")
print("  - Task: Identification with Pydantic output")
print("  - Crew: Sequential process")


**Key Points:**

1. **output_pydantic=ResearchItems**: This instructs the LLM to output structured data matching our Pydantic model
2. **Sequential Process**: Single agent, single task - runs sequentially
3. **Structured Output**: The output will be a Python object we can iterate over

**Why Pydantic Output?**
- Ensures consistent data structure
- Enables type checking and validation
- Makes it easy to pass data to the next stage (parallel research)
- Reduces parsing errors


## Building the Research Crew

### Step 4: Set Up RAG Tools

Before creating the research crew, we need to set up the tools that will enable RAG (Retrieval-Augmented Generation). These tools allow agents to search the web and access knowledge bases.


In [ ]:
# Set up RAG tools for research
# Note: These are CrewAI tools that provide web search and knowledge base access

# Web search tool - searches the internet for current information
do_web_search = WebsiteSearchTool(
    website="https://www.google.com/search?q="
)

# Knowledge base tool - searches local documents/files
# You can customize this to point to your knowledge base
do_knowledge_search = FileReadTool()

# Alternative: If you have a directory of documents
# do_knowledge_search = DirectoryReadTool(directory="./knowledge_base")

print("✅ RAG tools configured!")
print("\nTools available:")
print("  - do_web_search: Search the web for information")
print("  - do_knowledge_search: Search local documents/files")

# Note: You can add more tools here, such as:
# - Database search tools
# - API integration tools
# - Custom research tools


**About CrewAI Tools:**

CrewAI provides several built-in tools:
- `WebsiteSearchTool`: Web search capabilities
- `FileReadTool`: Read and search local files
- `DirectoryReadTool`: Search directories of documents
- Custom tools: You can create your own tools

**Tool Usage:**
- Agents automatically decide when to use tools
- Tools are called based on task requirements
- Results are incorporated into agent responses


### Step 5: Create the Research Agent and Crew

For each identified research item, a separate Research agent will run asynchronously, using RAG to search the web and user documents.

**Key Features:**
- **Parallel Execution**: Multiple research tasks run simultaneously
- **RAG Integration**: Each agent uses web search and knowledge base tools
- **Focused Research**: Each agent handles one specific research topic

**Inputs:**
- List of research items from the Research Item Identifier

**Outputs:**
- Detailed information, including references and explanations, for each item


In [ ]:
# Create the Researcher Agent
researcher = Agent(
    role='Researcher',
    goal='Conduct detailed research for each identified concept.',
    backstory="""You are an expert researcher with access to web search and 
    knowledge bases. You conduct thorough research, cite sources, and provide 
    comprehensive information on any topic. You always include in-line citations 
    for every claim you make.""",
    tools=[do_web_search, do_knowledge_search],
    llm=llm,
    verbose=True
)

# Create the research task
# Note: The task description uses placeholders that will be filled when tasks are created
research_task = Task(
    description="""Conduct thorough research on the following topic:
    
    Item: {item_name}
    Research Instructions: {research_instructions}
    
    Requirements:
    1. Use web search to find current, accurate information
    2. Search knowledge bases if relevant documents are available
    3. Provide detailed explanations with in-line citations
    4. Include references to all sources used
    5. Focus on answering the specific research instructions provided""",
    agent=researcher,
    expected_output='Comprehensive information that directly answers the research instructions, with in-line citations for every reference and a list of sources at the end.'
)

# Create the research crew
research_crew = Crew(
    agents=[researcher],
    tasks=[research_task],
    verbose=True
)

print("✅ Research Crew created!")
print("\nComponents:")
print("  - Agent: Researcher (with web search and knowledge base tools)")
print("  - Task: Research with dynamic placeholders")
print("  - Crew: Ready for parallel execution")


**Understanding Parallel Execution:**

The `kickoff_for_each_async()` method allows us to:
1. Take a list of research items
2. Create multiple tasks (one per item)
3. Execute all tasks in parallel
4. Collect all results

This is much faster than sequential execution when researching multiple topics!


## Putting It All Together

### Step 6: Complete Image Research Function

Now let's create a complete function that orchestrates the entire workflow:

1. **Image Analysis**: Use vision model to describe the image
2. **Item Identification**: Identify researchable topics
3. **Parallel Research**: Research all topics simultaneously
4. **Synthesis**: Combine findings into a comprehensive report


In [ ]:
def research_image(image_path: str, user_query: str = "Research all interesting aspects of this image.") -> dict:
    """
    Complete image research workflow.
    
    Args:
        image_path: Path to the image file
        user_query: User's research goal or question
        
    Returns:
        Dictionary containing:
            - image_description: Description from vision model
            - research_items: List of identified research topics
            - research_results: Results from parallel research
            - final_report: Synthesized comprehensive report
    """
    print("🔍 Starting image research workflow...")
    print(f"📷 Image: {image_path}")
    print(f"❓ Query: {user_query}\n")
    
    # Step 1: Describe the image using vision model
    print("Step 1/4: Analyzing image with vision model...")
    try:
        image_description = describe_image_ollama_direct(
            image_path, 
            "Describe the image in detail for educational purposes."
        )
        print(f"✅ Image analyzed ({len(image_description)} characters)\n")
    except Exception as e:
        print(f"❌ Error analyzing image: {e}")
        return {"error": f"Image analysis failed: {e}"}
    
    # Step 2: Identify research items
    print("Step 2/4: Identifying research topics...")
    try:
        result = identifier_crew.kickoff(
            inputs={
                "image_description": image_description,
                "user_query": user_query
            }
        )
        
        # Extract research items from the result
        research_items = identification_task.output.pydantic.items
        print(f"✅ Identified {len(research_items)} research topics:\n")
        for i, item in enumerate(research_items, 1):
            print(f"  {i}. {item.item_name}")
        print()
    except Exception as e:
        print(f"❌ Error identifying research items: {e}")
        return {"error": f"Research identification failed: {e}"}
    
    # Step 3: Parallel research
    print("Step 3/4: Conducting parallel research...")
    try:
        # Prepare tasks for parallel execution
        tasks = []
        for item in research_items:
            tasks.append({
                "item_name": item.item_name,
                "research_instructions": item.research_instructions
            })
        
        # Execute research in parallel
        research_results = research_crew.kickoff_for_each_async(tasks)
        print(f"✅ Research completed for {len(tasks)} topics\n")
    except Exception as e:
        print(f"❌ Error during research: {e}")
        return {"error": f"Research failed: {e}"}
    
    # Step 4: Synthesize findings
    print("Step 4/4: Synthesizing findings into comprehensive report...")
    try:
        # Combine all research results
        gathered_info = "\n\n".join([
            f"Topic: {task['item_name']}\nResearch: {result}"
            for task, result in zip(tasks, research_results)
        ])
        
        # Create synthesis prompt
        synthesis_prompt = f"""Thoroughly answer the user's question, citing all references.

User Query: {user_query}

Image Description: {image_description}

Gathered Information:
{gathered_info}

Please synthesize this information into a comprehensive, well-organized report that:
1. Directly answers the user's query
2. Incorporates all research findings
3. Maintains all citations and references
4. Provides a clear, educational summary"""
        
        # Generate final report
        final_report = llm.call(synthesis_prompt)
        print("✅ Report synthesized\n")
        
        return {
            "image_description": image_description,
            "research_items": [
                {"item_name": item.item_name, "research_instructions": item.research_instructions}
                for item in research_items
            ],
            "research_results": research_results,
            "final_report": final_report
        }
    except Exception as e:
        print(f"❌ Error synthesizing report: {e}")
        return {"error": f"Report synthesis failed: {e}"}


print("✅ Complete image research function created!")
print("\nFunction: research_image(image_path, user_query)")
print("This function orchestrates the entire workflow from image to final report.")


### Example Usage

Here's how to use the complete system:

```python
# Example 1: Research a historical photo
result = research_image(
    image_path="statue_of_liberty.jpg",
    user_query="Tell me about the history of the Statue of Liberty and anything else interesting in this image."
)

# Access the results
print("Final Report:")
print(result["final_report"])

# Example 2: Analyze an architecture diagram
result = research_image(
    image_path="system_architecture.png",
    user_query="Explain the components and their relationships in this architecture diagram."
)
```

**Note**: Make sure you have an image file ready to test with. You can use any image format supported by the vision model (PNG, JPEG, etc.).


## Advanced Techniques

### 1. Custom Research Tools

You can create custom tools for specialized research:

```python
from crewai_tools import tool

@tool("Database Search")
def search_database(query: str) -> str:
    """Search internal database for information."""
    # Your database search logic here
    return results

# Add to researcher agent
researcher = Agent(
    role='Researcher',
    tools=[do_web_search, do_knowledge_search, search_database],
    # ... other parameters
)
```

### 2. Filtering Research Items

You can add logic to filter or prioritize research items:

```python
def filter_research_items(items: List[ResearchItem], max_items: int = 5) -> List[ResearchItem]:
    """Filter and prioritize research items."""
    # Add your filtering logic here
    # For example, prioritize items mentioned in user query
    return items[:max_items]
```

### 3. Caching Image Descriptions

To avoid re-analyzing the same images:

```python
import hashlib
import json

image_cache = {}

def get_cached_description(image_path: str) -> str:
    """Get cached image description if available."""
    # Create hash of image
    with open(image_path, 'rb') as f:
        image_hash = hashlib.md5(f.read()).hexdigest()
    
    if image_hash in image_cache:
        return image_cache[image_hash]
    
    # Generate and cache
    description = describe_image_ollama_direct(image_path)
    image_cache[image_hash] = description
    return description
```

### 4. Streaming Research Results

For real-time updates during research:

```python
def research_with_streaming(image_path: str, user_query: str):
    """Research with streaming updates."""
    # Your streaming implementation
    # Use async generators or callbacks
    pass
```

### 5. Multi-Language Support

Add language detection and translation:

```python
from langdetect import detect

def detect_and_translate(text: str, target_language: str = "en") -> str:
    """Detect language and translate if needed."""
    detected = detect(text)
    if detected != target_language:
        # Translate using your translation service
        pass
    return text
```


## Edge Cases and Error Handling

### Common Edge Cases

Let's handle various edge cases that might occur:


In [ ]:
def robust_research_image(image_path: str, user_query: str = "Research all interesting aspects of this image.") -> dict:
    """
    Robust version with comprehensive error handling.
    """
    errors = []
    warnings = []
    
    # Validate inputs
    if not os.path.exists(image_path):
        return {"error": f"Image file not found: {image_path}"}
    
    if not user_query or len(user_query.strip()) == 0:
        user_query = "Research all interesting aspects of this image."
        warnings.append("Empty user query, using default")
    
    # Check file size (very large images might cause issues)
    file_size = os.path.getsize(image_path)
    if file_size > 10 * 1024 * 1024:  # 10MB
        warnings.append(f"Large image file ({file_size / 1024 / 1024:.2f}MB), processing may be slow")
    
    try:
        # Step 1: Image analysis with retry logic
        max_retries = 3
        image_description = None
        
        for attempt in range(max_retries):
            try:
                image_description = describe_image_ollama_direct(image_path)
                if image_description and len(image_description) > 50:
                    break
            except Exception as e:
                if attempt == max_retries - 1:
                    return {"error": f"Failed to analyze image after {max_retries} attempts: {e}"}
                print(f"Retry {attempt + 1}/{max_retries} for image analysis...")
        
        if not image_description:
            return {"error": "Image analysis returned empty description"}
        
        # Step 2: Research identification with fallback
        try:
            result = identifier_crew.kickoff(
                inputs={
                    "image_description": image_description,
                    "user_query": user_query
                }
            )
            research_items = identification_task.output.pydantic.items
            
            if not research_items or len(research_items) == 0:
                warnings.append("No research items identified, using fallback")
                # Fallback: create a single research item
                research_items = [ResearchItem(
                    item_name="General Image Analysis",
                    research_instructions="Research the main subjects and context of this image."
                )]
        except Exception as e:
            errors.append(f"Research identification error: {e}")
            # Fallback
            research_items = [ResearchItem(
                item_name="General Image Analysis",
                research_instructions="Research the main subjects and context of this image."
            )]
        
        # Step 3: Parallel research with timeout handling
        research_results = []
        tasks = []
        
        for item in research_items:
            tasks.append({
                "item_name": item.item_name,
                "research_instructions": item.research_instructions
            })
        
        try:
            research_results = research_crew.kickoff_for_each_async(tasks)
            
            # Validate results
            if not research_results or len(research_results) != len(tasks):
                warnings.append("Some research tasks may have failed")
        except Exception as e:
            errors.append(f"Research execution error: {e}")
            # Continue with partial results
        
        # Step 4: Synthesis with fallback
        try:
            if research_results:
                gathered_info = "\n\n".join([
                    f"Topic: {task['item_name']}\nResearch: {result}"
                    for task, result in zip(tasks, research_results)
                ])
            else:
                gathered_info = "No research results available."
            
            synthesis_prompt = f"""Thoroughly answer the user's question, citing all references.

User Query: {user_query}

Image Description: {image_description}

Gathered Information:
{gathered_info}

Please synthesize this information into a comprehensive, well-organized report."""
            
            final_report = llm.call(synthesis_prompt)
        except Exception as e:
            errors.append(f"Report synthesis error: {e}")
            final_report = f"Error generating report: {e}\n\nImage Description: {image_description}"
        
        return {
            "image_description": image_description,
            "research_items": [
                {"item_name": item.item_name, "research_instructions": item.research_instructions}
                for item in research_items
            ],
            "research_results": research_results if research_results else [],
            "final_report": final_report,
            "warnings": warnings,
            "errors": errors
        }
        
    except Exception as e:
        return {"error": f"Unexpected error: {e}"}


print("✅ Robust research function with error handling created!")
print("\nThis version handles:")
print("  - Missing or invalid image files")
print("  - Empty or invalid queries")
print("  - Large image files")
print("  - Network/timeout errors")
print("  - Empty research results")
print("  - Partial failures")


### Edge Case: No Research Items Found

**Problem**: Vision model identifies image but no researchable items are found.

**Solution**: Implement fallback logic to create a general research item.

### Edge Case: Research Timeout

**Problem**: Some research tasks take too long or fail.

**Solution**: Implement timeout handling and continue with partial results.

### Edge Case: Empty Image Description

**Problem**: Vision model returns empty or very short description.

**Solution**: Add validation and retry logic with different prompts.

### Edge Case: Network Issues

**Problem**: Web search or API calls fail due to network issues.

**Solution**: Implement retry logic and graceful degradation.


## Troubleshooting Guide

### Common Issues and Solutions

#### Issue 1: Ollama Connection Error

**Symptoms:**
```
ConnectionError: Cannot connect to Ollama
```

**Solutions:**
1. Verify Ollama is running:
   ```bash
   ollama serve
   ```
2. Check if Ollama is accessible:
   ```bash
   curl http://localhost:11434/api/tags
   ```
3. Verify the base URL in your code matches your Ollama setup
4. Check firewall settings if using remote Ollama

#### Issue 2: Model Not Found

**Symptoms:**
```
Error: model 'granite3.2:8b-instruct-q8_0' not found
```

**Solutions:**
1. Pull the required models:
   ```bash
   ollama pull granite3.2:8b-instruct-q8_0
   ollama pull granite3.2-vision:2b
   ```
2. Verify models are available:
   ```bash
   ollama list
   ```
3. Check model names match exactly (case-sensitive)

#### Issue 3: Vision Model Not Working

**Symptoms:**
- Image description is empty or generic
- Vision model call fails

**Solutions:**
1. Use the direct Ollama API method (`describe_image_ollama_direct`)
2. Verify image format is supported (PNG, JPEG)
3. Check image file size (very large images may timeout)
4. Try a simpler prompt first
5. Verify vision model is properly pulled:
   ```bash
   ollama show granite3.2-vision:2b
   ```

#### Issue 4: Pydantic Validation Errors

**Symptoms:**
```
ValidationError: ResearchItems validation failed
```

**Solutions:**
1. Check that the LLM output matches the expected structure
2. Add more detailed instructions in the task description
3. Use examples in the prompt to guide the LLM
4. Consider using a more capable model (q8_0 instead of q4_K_M)

#### Issue 5: Research Tasks Failing

**Symptoms:**
- Some research tasks return empty results
- Web search not working

**Solutions:**
1. Verify internet connection
2. Check if web search tool is properly configured
3. Add retry logic for failed tasks
4. Verify CrewAI tools are properly installed:
   ```bash
   pip install --upgrade crewai-tools
   ```

#### Issue 6: Memory Issues

**Symptoms:**
- Out of memory errors
- Slow performance

**Solutions:**
1. Reduce number of parallel research tasks
2. Use smaller quantization (q4_K_M instead of q8_0)
3. Process images in batches
4. Clear cache between runs
5. Close other applications using memory

#### Issue 7: Slow Performance

**Symptoms:**
- Research takes very long
- Timeouts occur

**Solutions:**
1. Reduce number of research items
2. Use GPU acceleration if available
3. Optimize image size before processing
4. Use faster models or quantization
5. Implement caching for repeated queries

### Debugging Tips

1. **Enable Verbose Mode**: Set `verbose=True` in Crew and Agent definitions
2. **Check Logs**: Monitor Ollama logs and CrewAI output
3. **Test Components Separately**: Test vision model, then identification, then research
4. **Use Simple Test Cases**: Start with simple images and queries
5. **Validate Inputs**: Always validate image paths and user queries

### Getting Help

If you encounter issues not covered here:

1. Check CrewAI documentation: https://docs.crewai.com
2. Check Ollama documentation: https://ollama.ai
3. Review error messages carefully - they often contain clues
4. Test with minimal examples to isolate the problem
5. Check GitHub issues for CrewAI and Ollama


## Exercises

### Exercise 1: Basic Image Research

**Goal**: Get familiar with the basic workflow.

**Tasks:**
1. Find or create a simple image (e.g., a photo of a famous landmark)
2. Use `research_image()` to analyze it
3. Print the final report
4. Examine the research items that were identified

**Expected Outcome**: You should see:
- Image description from vision model
- List of identified research topics
- Research results for each topic
- Final synthesized report

**Challenge**: Try with different types of images (diagram, photo, chart, etc.)

---

### Exercise 2: Custom Research Query

**Goal**: Learn to customize research queries.

**Tasks:**
1. Use the same image from Exercise 1
2. Create a specific research query (e.g., "Focus on the historical significance")
3. Compare results with the default query
4. Analyze how the query affects research items identified

**Expected Outcome**: Different research items based on your query focus.

**Challenge**: Try very specific vs. very general queries and compare results.

---

### Exercise 3: Error Handling

**Goal**: Practice handling edge cases.

**Tasks:**
1. Try the function with:
   - A non-existent image path
   - An invalid image file
   - An empty query
   - A very large image file
2. Implement error handling for each case
3. Add user-friendly error messages

**Expected Outcome**: Graceful error handling without crashes.

**Challenge**: Create a wrapper function that validates all inputs before processing.

---

### Exercise 4: Custom Research Tools

**Goal**: Add custom research capabilities.

**Tasks:**
1. Create a custom tool that searches a specific website (e.g., Wikipedia)
2. Add it to the researcher agent
3. Compare results with and without the custom tool

**Expected Outcome**: Enhanced research results using your custom tool.

**Challenge**: Create a tool that searches multiple sources and combines results.

---

### Exercise 5: Research Item Filtering

**Goal**: Optimize research by filtering items.

**Tasks:**
1. Modify the identification step to limit research items to top 3 most important
2. Add logic to prioritize items mentioned in user query
3. Compare research quality with filtered vs. unfiltered items

**Expected Outcome**: Faster research with focused, relevant results.

**Challenge**: Implement smart filtering based on item importance scores.

---

### Exercise 6: Batch Processing

**Goal**: Process multiple images efficiently.

**Tasks:**
1. Create a function to process multiple images
2. Implement parallel processing for multiple images
3. Generate a summary report for all images

**Expected Outcome**: Efficient batch processing of image collections.

**Challenge**: Add progress tracking and resume capability for large batches.

---

### Exercise 7: Research Quality Metrics

**Goal**: Evaluate research quality.

**Tasks:**
1. Create metrics to evaluate research quality:
   - Number of sources cited
   - Length of research results
   - Relevance to query
2. Implement a scoring system
3. Compare different research runs

**Expected Outcome**: Quantitative measures of research quality.

**Challenge**: Create a feedback loop to improve research based on scores.

---

### Exercise 8: Integration with Knowledge Base

**Goal**: Integrate local document search.

**Tasks:**
1. Create a knowledge base directory with relevant documents
2. Configure `do_knowledge_search` to use your knowledge base
3. Compare research results with and without knowledge base
4. Analyze when knowledge base results are preferred over web search

**Expected Outcome**: Research that combines web and local knowledge.

**Challenge**: Implement smart routing (when to use web vs. knowledge base).

---

### Exercise 9: Multi-Language Support

**Goal**: Support research in multiple languages.

**Tasks:**
1. Detect the language of user query
2. Translate research instructions if needed
3. Research in the appropriate language
4. Translate results back if needed

**Expected Outcome**: Research agent that works in multiple languages.

**Challenge**: Maintain citations and references across translations.

---

### Exercise 10: Real-Time Research Dashboard

**Goal**: Create a monitoring interface.

**Tasks:**
1. Add progress tracking to the research function
2. Create a simple dashboard showing:
   - Current step
   - Research items being processed
   - Progress percentage
3. Display results as they come in

**Expected Outcome**: Visual feedback during research process.

**Challenge**: Add real-time updates using web sockets or similar technology.

---

### Bonus Exercise: Complete Application

**Goal**: Build a complete, production-ready application.

**Tasks:**
1. Create a web interface (Flask/FastAPI)
2. Add image upload functionality
3. Implement user authentication
4. Add research history and caching
5. Create a results display page
6. Add export functionality (PDF, Markdown, etc.)

**Expected Outcome**: A complete, deployable image research application.

**Challenge**: Add advanced features like research templates, scheduled research, and API endpoints.


## Summary

Congratulations! You've learned how to build a comprehensive AI research agent for image analysis. Let's recap what we covered:

### Key Concepts Learned

1. **Vision Models**: Using Granite Vision to analyze and describe images
2. **Agentic AI**: Orchestrating multiple AI agents with CrewAI
3. **RAG (Retrieval-Augmented Generation)**: Combining web search and knowledge bases with LLMs
4. **Parallel Processing**: Running multiple research tasks simultaneously
5. **Structured Output**: Using Pydantic for type-safe data handling
6. **Error Handling**: Building robust systems that handle edge cases

### Architecture Overview

```
Image Input
    ↓
Vision Model (Granite 3.2 Vision)
    ↓
Image Description
    ↓
Item Identifier Agent
    ↓
Research Items (Structured List)
    ↓
Parallel Research Agents (with RAG tools)
    ↓
Research Results
    ↓
Synthesis Agent
    ↓
Final Comprehensive Report
```

### Components Built

1. **Vision Integration**: Functions to process images and get descriptions
2. **Item Identifier Agent**: Identifies researchable topics from images
3. **Research Crew**: Parallel research agents with RAG capabilities
4. **Complete Workflow**: End-to-end image research function
5. **Error Handling**: Robust version with comprehensive error handling

### Best Practices

1. ✅ **Always validate inputs** before processing
2. ✅ **Use structured output** (Pydantic) for reliable data flow
3. ✅ **Implement error handling** for all external calls
4. ✅ **Enable verbose mode** during development
5. ✅ **Test components separately** before integration
6. ✅ **Cache results** when possible to improve performance
7. ✅ **Monitor resource usage** (memory, API calls, etc.)

### Next Steps

1. **Experiment**: Try different images and queries
2. **Customize**: Add your own tools and agents
3. **Optimize**: Improve performance and quality
4. **Deploy**: Build a production application
5. **Extend**: Add features like multi-language support, caching, etc.

### Resources

- **CrewAI Documentation**: https://docs.crewai.com
- **Ollama Documentation**: https://ollama.ai
- **Granite Models**: https://github.com/ibm-granite-community
- **Pydantic Documentation**: https://docs.pydantic.dev

### Final Thoughts

You now have a powerful image research agent that can:
- Analyze any image
- Identify researchable topics
- Conduct parallel research using RAG
- Synthesize findings into comprehensive reports

This system demonstrates the power of combining:
- **Vision AI** for image understanding
- **Agentic AI** for orchestration
- **RAG** for information retrieval
- **LLMs** for synthesis

The possibilities are endless - from educational tools to research assistants to content analysis systems. Keep experimenting and building!

---

**Happy Building! 🚀**
